### train

In [1]:
import json, base64
params = {
    "model_id": "20231026000000",
    "start_time": "2023-09-01",
    "end_time": "2023-10-10",
    "model_type": 0
}
params1 = json.dumps(params, indent=4)

b = params1.encode("UTF-8")
e = base64.b64encode(b)
argv = e.decode("UTF-8")
print(f'python train.py "{argv}"')

python train.py "ewogICAgIm1vZGVsX2lkIjogIjIwMjMxMDI2MDAwMDAwIiwKICAgICJzdGFydF90aW1lIjogIjIwMjMtMDktMDEiLAogICAgImVuZF90aW1lIjogIjIwMjMtMTAtMTAiLAogICAgIm1vZGVsX3R5cGUiOiAwCn0="


### predict

In [2]:
import pandas as pd
df = pd.read_excel(r"C:\Users\tzuli\Documents\python\hosong\data\train\train.xlsx",
                   sheet_name="simulation", header=[0, 1, 2, 3])
df = df.iloc[:, 1:]

# find ER samples
ER_col = [i for i in df.columns if i[3] == "ER"]
ok = df[(df[ER_col] == 0).all(axis = 1)].reset_index(drop = True)
df = df[(df[ER_col] != 0).any(axis = 1)].reset_index(drop = True)

# calculate total ER by each side
left_ER_col = [i for i in df.columns if (i[3] == "ER") and (i[1] == "左邊")]
right_ER_col = [i for i in df.columns if (i[3] == "ER") and (i[1] == "右邊")]
left = df[left_ER_col].sum(axis=1)
right = df[right_ER_col].sum(axis=1)

# calculate difference between left and right
error = abs(left - right)

# rename columns
df.columns = [(cols[2] + "_" + cols[3]) for cols in df.columns]
ok.columns = [(cols[2] + "_" + cols[3]) for cols in ok.columns]

# choose the adjustment method
ng1_3 = df[error > 0.001]
ng4 = df[(error <= 0.001) & (left < right)]
ng5 = df[(error <= 0.001) & (left >= right)]

params = ng1_3.sample(1).iloc[0].to_dict() # ok, ng1_3, ng4, ng5

params = df[(left < right)].sample(1).iloc[0].to_dict()

In [3]:
with open(r"C:\Users\tzuli\Documents\python\hosong\data\predict\input.json", 'w', encoding='utf-8') as file:
    json.dump(params, file, indent=4, ensure_ascii=False)

In [4]:
import json, base64
from datetime import datetime
params = {
    "model_id": ["20231025000000", "20231026000000"],
    "upload_time":  datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "machine_id": "XXXX",
    "path": r"C:\Users\tzuli\Documents\python\hosong\data\predict"
}
params1 = json.dumps(params, indent=4)

b = params1.encode("UTF-8")
e = base64.b64encode(b)
argv = e.decode("UTF-8")
print(f'python predict.py "{argv}"')

python predict.py "ewogICAgIm1vZGVsX2lkIjogWwogICAgICAgICIyMDIzMTAyNTAwMDAwMCIsCiAgICAgICAgIjIwMjMxMDI2MDAwMDAwIgogICAgXSwKICAgICJ1cGxvYWRfdGltZSI6ICIyMDIzLTEwLTMxIDEyOjA3OjUyIiwKICAgICJtYWNoaW5lX2lkIjogIlhYWFgiLAogICAgInBhdGgiOiAiQzpcXFVzZXJzXFx0enVsaVxcRG9jdW1lbnRzXFxweXRob25cXGhvc29uZ1xcZGF0YVxccHJlZGljdCIKfQ=="
